In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.contrib import rnn
import itertools as ittls
from collections import Counter

In [2]:
## generate batch index for loop interation
def batch_gen(batch_size,polulation_size=500):
    collection = []
    for _ in ittls.cycle(xrange(polulation_size)):
        collection.append(_)
        if len(collection) >= batch_size:
            yield collection
            collection = []

def mnist_encoder(y):
    encoded_y = np.zeros((len(y),10),dtype=np.float32)
    for index,label in enumerate(y):
        encoded_y[index,label] = 1.0
    return encoded_y

In [8]:
class basic_LSTM(object):
    def __init__(self, sess, time_step, input_dim, hidden_size, class_num, batch_size):
        self.time_step = time_step
        self.input_dim = input_dim
        self.hidden_size = hidden_size
        self.class_num = class_num
        self.batch_size = batch_size
        self.rnn_cell = tf.contrib.rnn.BasicLSTMCell(hidden_size)
        
        self.input_x = tf.placeholder(dtype = tf.float32, shape = [None,time_step,input_dim])
        self.label_y = tf.placeholder(dtype = tf.float32, shape = [None, class_num])
        
        state = self.rnn_cell.zero_state(batch_size = self.batch_size, dtype = tf.float32)
        
        self.output_list = []
        for each_time_step in tf.unstack(self.input_x, time_step, axis=1):
            output, state = self.rnn_cell(each_time_step, state)
            self.output_list.append(output)
        self.softmax_weight = tf.Variable(tf.random_normal([self.hidden_size,self.class_num]))
        self.softmax_bias = tf.Variable(tf.random_normal([self.class_num]))
        logits = tf.matmul(self.output_list[-1],self.softmax_weight) + self.softmax_bias
        
        self.prediction = tf.nn.softmax(logits)
        self.cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits,labels=self.label_y))
        
        correct_pred = tf.equal(tf.argmax(self.prediction,1),tf.argmax(self.label_y,1))
        self.accuray = tf.reduce_mean(tf.cast(correct_pred,tf.float32))
        
        sess.run(tf.global_variables_initializer())
        self.trained = False
        
    def batch_gen(self,batch_size,polulation_size=500):
        collection = []
        for _ in ittls.cycle(xrange(polulation_size)):
            collection.append(_)
            if len(collection) >= batch_size:
                yield collection
                collection = []

    def fit(self, sess, x, y, learning_rate = 0.1,iterations=1000):
        """training process: create optimizer and perform training loop"""
        assert x.shape[1:] == self.input_x.shape[1:]
        
        optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
        train_op = optimizer.minimize(self.cost)
        
        index_gen = self.batch_gen(batch_size = self.batch_size, polulation_size=x.shape[0])
        cost_list = []
        for step in xrange(1,iterations+1):
            index = next(index_gen)
            feed_x = x[index]
            feed_y = y[index]
            cost,_ = sess.run([self.cost,train_op],feed_dict={self.input_x:feed_x, self.label_y:feed_y})
            cost_list.append(cost)
            if step % 200 == 0 or step==1 or step==iterations:
                cost,acc = sess.run([self.cost,self.accuray],
                                    feed_dict={self.input_x:feed_x, self.label_y:feed_y})
                print "step:",step,",batch_cost=","{:.4f}".format(cost),",accuray=","{:.3f}".format(acc)
        self.trained = True
        
    def predict(sess, x):
        pass
with tf.Graph().as_default():
    with tf.Session() as sess:
        x = np.random.random((1000,10,15))
        y = np.random.binomial(n=1, p=0.4, size=(1000,1))
        encoded_y = np.zeros((len(y),2),dtype=np.float32)
        for index,label in enumerate(y):
            encoded_y[index,label] = 1.0
        print y.shape
        lstm = basic_LSTM(sess=sess, time_step= 10, input_dim = 15, hidden_size = 200, class_num = 2, batch_size = 50)
        lstm.fit(sess=sess, x=x, y=encoded_y, learning_rate = 0.001,iterations=5000)

(1000, 1)
step: 1 ,batch_cost= 0.8882 ,accuray= 0.340
step: 200 ,batch_cost= 0.6731 ,accuray= 0.580
step: 400 ,batch_cost= 0.6656 ,accuray= 0.560
step: 600 ,batch_cost= 0.6605 ,accuray= 0.580
step: 800 ,batch_cost= 0.6568 ,accuray= 0.580
step: 1000 ,batch_cost= 0.6540 ,accuray= 0.580
step: 1200 ,batch_cost= 0.6519 ,accuray= 0.580
step: 1400 ,batch_cost= 0.6502 ,accuray= 0.580
step: 1600 ,batch_cost= 0.6488 ,accuray= 0.600
step: 1800 ,batch_cost= 0.6477 ,accuray= 0.620
step: 2000 ,batch_cost= 0.6466 ,accuray= 0.640
step: 2200 ,batch_cost= 0.6458 ,accuray= 0.640
step: 2400 ,batch_cost= 0.6450 ,accuray= 0.640
step: 2600 ,batch_cost= 0.6443 ,accuray= 0.640
step: 2800 ,batch_cost= 0.6436 ,accuray= 0.620
step: 3000 ,batch_cost= 0.6431 ,accuray= 0.620
step: 3200 ,batch_cost= 0.6425 ,accuray= 0.620
step: 3400 ,batch_cost= 0.6421 ,accuray= 0.620
step: 3600 ,batch_cost= 0.6416 ,accuray= 0.620
step: 3800 ,batch_cost= 0.6412 ,accuray= 0.620
step: 4000 ,batch_cost= 0.6409 ,accuray= 0.620
step: 4200

In [18]:
s1 = '142857'
s2 = '769230'